<a href="https://colab.research.google.com/github/kunai-3txk/student-compe/blob/main/PSP_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ref
https://www.kaggle.com/code/kentaro7/beginner-lgbm

https://www.kaggle.com/code/mohammad2012191/lgbm-early-stopping-lb-0-670

# import

In [1]:
!pip install polars
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd
import polars as pl
import os
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import GroupKFold, KFold
from catboost import CatBoostClassifier, Pool
import matplotlib.pyplot as plt
import warnings
from itertools import combinations
import math
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 200)



In [26]:
from sklearn.model_selection import KFold, GroupKFold
from lightgbm.sklearn import LGBMRegressor
from sklearn.metrics import f1_score

## Config

In [3]:
class paths:
    # kaggle環境ならTrue
    if 'KAGGLE_URL_BASE' in set(os.environ.keys()):
        common_path = "/kaggle/input/amp-parkinsons-disease-progression-prediction"
    
    # colaboratory環境ならTrue
    if 'COLAB_GPU' in set(os.environ.keys()):
        common_path = "/content/drive/MyDrive/PSP/"

    SAMPLE_SUBMISSION = common_path + "sample_submission.csv"
    TRAIN = common_path + "train.csv"
    TRAIN_LABELS = common_path + "train_labels.csv"
    TEST = common_path + "test.csv"
    OUTPUT = common_path

# defnition

In [4]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

# Load Data

In [17]:
targets = pd.read_csv(paths.TRAIN_LABELS)
train_ori  =  reduce_mem_usage(pd.read_csv(paths.TRAIN))
test_ori =  pd.read_csv(paths.TEST)
sample=sample_submission =  pd.read_csv(paths.SAMPLE_SUBMISSION)

Memory usage of dataframe is 2010.23 MB
Memory usage after optimization is: 778.99 MB
Decreased by 61.2%


In [19]:
targets['session'] = targets.session_id.apply(lambda x: int(x.split('_')[0]))
targets['q'] = targets.session_id.apply(lambda x: int(x.split('_')[-1][1:]))

train = train_ori.copy()
test = test_ori.copy()
train.drop(["fullscreen","hq","music"],axis=1,inplace=True)

#train

In [6]:
#train.describe()
#train.info()

In [7]:
from sklearn.preprocessing import LabelEncoder

In [21]:
def feature_engineer(data):

  data["elapsed_time"].fillna(data["elapsed_time"].mean())
  data["elapsed_time"] = np.log(data["elapsed_time"]).round(3)
  data["elapsed_time"] = data["elapsed_time"].replace([np.inf, -np.inf], np.nan)
  data["elapsed_time"].fillna(2.25,inplace=True)

  le = LabelEncoder()

  data["event_name"] = le.fit_transform(data["event_name"])
  data["name"] = le.fit_transform(data["name"])
  data["page"] = data["page"].fillna(-1)
  data["page"] = data["page"].astype(int)
  data["room_coor_x"].fillna(0,inplace=True)
  data["room_coor_y"].fillna(0,inplace=True)
  data["screen_coor_x"].fillna(0,inplace=True)
  data["screen_coor_y"].fillna(0,inplace=True)
  data["hover_duration"].fillna(0,inplace=True)

  data.reset_index(drop=True)

  KEYS = ['session_id','level_group']
  NUMS = ['elapsed_time','room_coor_x', 'room_coor_y', 'screen_coor_x', 'screen_coor_y', 'hover_duration']
  CATS = ['event_name','name','fqid', 'room_fqid', 'text_fqid']

  data_byn = data.groupby(KEYS)[NUMS].agg(["max","min","mean","std"])
  data_byn.columns = ['_'.join(col) for col in data_byn.columns]
  data_byn = data_byn.reset_index()

  data_byc = data.groupby(KEYS)[CATS].agg('nunique')
  data_byc = data_byc.reset_index()
  #train_byc = train_byc.set_index("session_id")

  ret_data =  data_byn.merge(data_byc,on=['session_id','level_group'])

  ret_data =  ret_data.reset_index(drop=True)
  ret_data = ret_data.set_index("session_id")

  #print('byn shape:',data_byn.shape,'byc shape:',data_byc.shape,'ret_df shape:',ret_data.shape)
  return ret_data

In [24]:
df_train = feature_engineer(train)
print(df_train.shape)
df_train.head()

(35337, 30)


In [25]:
df_test = feature_engineer(test)
print(df_test.shape)
df_test.head()

(9, 30)


In [12]:
FEATURES = [c for c in df_train.columns if c != 'level_group_']
print(FEATURES)
ALL_USERS = df_train.index.unique()
print(ALL_USERS)

['index', 'level_group', ('session_id', ''), ('level_group', ''), ('elapsed_time', 'max'), ('elapsed_time', 'min'), ('elapsed_time', 'mean'), ('elapsed_time', 'std'), ('room_coor_x', 'max'), ('room_coor_x', 'min'), ('room_coor_x', 'mean'), ('room_coor_x', 'std'), ('room_coor_y', 'max'), ('room_coor_y', 'min'), ('room_coor_y', 'mean'), ('room_coor_y', 'std'), ('screen_coor_x', 'max'), ('screen_coor_x', 'min'), ('screen_coor_x', 'mean'), ('screen_coor_x', 'std'), ('screen_coor_y', 'max'), ('screen_coor_y', 'min'), ('screen_coor_y', 'mean'), ('screen_coor_y', 'std'), ('hover_duration', 'max'), ('hover_duration', 'min'), ('hover_duration', 'mean'), ('hover_duration', 'std'), 'event_name', 'name', 'fqid', 'room_fqid', 'text_fqid']
Int64Index([20090312431273200, 20090312433251036, 20090314121766812,
            20090314363702160, 20090314441803444, 20090315081004164,
            20090315085850788, 20090315101457836, 20090315170769824,
            20090317080721164,
            ...
          

#fit

In [27]:
N_FOLDS = 10

gkf = GroupKFold(n_splits=N_FOLDS)
oof = pd.DataFrame(data=np.zeros((len(ALL_USERS),18)), index=ALL_USERS)
models = {}

# COMPUTE CV SCORE WITH N GROUP K FOLD
for i, (train_index, test_index) in enumerate(gkf.split(X=df_train, groups=df_train.index)):
    print('#'*25)
    print('### Fold',i+1)
    print('#'*25)
    
    # ITERATE THRU QUESTIONS 1 THRU 18
    for t in range(1,19):
        
        # USE THIS TRAIN DATA WITH THESE QUESTIONS
        if t<=3: grp = "0-4"
        elif t<=13: grp = "5-12"
        elif t<=22: grp = "13-22"
            
        # TRAIN DATA
        train_x = df_train.iloc[train_index]
        train_x = train_x.loc[train_x[level_group] == grp]
        train_users = train_x.index.values
        train_y = targets.loc[targets[q]==t].set_index('session').loc[train_users]
        
        # VALID DATA
        valid_x = df_train.iloc[test_index]
        valid_x = valid_x.loc[valid_x.level_group_ == grp]
        valid_users = valid_x.index.values
        valid_y = targets.loc[targets.q==t].set_index('session').loc[valid_users]
        
        # TRAIN MODEL
        model = LGBMRegressor(learning_rate=0.027, \
                    num_leaves=15, \
                    n_estimators=200, \
                    min_child_samples=20, \
                    boosting_type='gbdt',
                    subsample_for_bin=1000,
                    max_depth=-1,
                    colsample_bytree=0.8)
        model.fit(train_x[FEATURES].astype('float32'), train_y['correct'])
        
        # SAVE MODEL, PREDICT VALID OOF
        models[f'{i}_{grp}_{t}'] = model
        oof.loc[valid_users, t-1] = model.predict(valid_x[FEATURES])
        
    print()

#########################
### Fold 1
#########################


AttributeError: ignored